Requirements:
- langchain
- langchain-openai
- pandas
- chromadb
- openai
- streamlit
- langchain-community
- openpyxl
- langchain-chroma

In [1]:
import langchain
import pandas as pd
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import os
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

from chatbot import setup_vector_store, setup_embedding_model, get_job_description_for_query, get_summary, get_output, get_summary_chain, get_output_prompt

from api_key import api_key

In [2]:
os.environ["OPENAI_API_KEY"] = api_key

#model = "text-embedding-3-small"
#embedding_model = OpenAIEmbeddings(model=model)

In [164]:
def createDocuments(df):
    documents = []
    for index, row in df.iterrows():
        document = Document(page_content=row["Beschreibung der Position und Zuständigkeiten bei Problemen"], metadata={"name":row["Name"], "personal_id": index})
        documents.append(document)
        document = Document(page_content=row.fillna("")["Betreute Programme"], metadata={"name": row["Name"], "personal_id": index})
        documents.append(document)
    return documents

In [165]:
path_to_xlsx = "/Users/annka/Downloads/KI-HackathonxROSSMANN_Challenge_Ansprechpartner-Chatbot.xlsx"
df = pd.read_excel(path_to_xlsx)
#job_descriptions = df["Beschreibung der Position und Zuständigkeiten bei Problemen"].tolist()
#programs = df["Betreute Programme"].fillna("").tolist()
documents = createDocuments(df)

#vector_store = Chroma.from_texts(documents, embedding=embedding_model)
vector_store = Chroma(embedding_function=embedding_model)
vector_store.add_documents(documents=documents)

['3eca9e1b-a286-4bce-a2cd-7ad2d88241a7',
 'f7d0bd86-1f54-4ddc-8e04-9e07a397f0a5',
 '1ef3a6c8-e4c6-42b8-896a-07ce7cbb32f2',
 'b8eba65b-4b7f-46bc-95c6-3b3d28c9dedd',
 'd8fdb977-b429-407e-9e30-588386f62cb8',
 '7c3f03f8-2901-4211-b0fe-408ffe9302e1',
 '360a6d99-ef2a-42f0-a1ab-f87de04a1cd1',
 '2db50b23-010b-4d70-b1b9-892860e62fa6',
 'e2d2d21f-5069-4aad-8b1c-dbcbdeb1654b',
 '8d745f91-71ab-40a4-9f64-2924ef83fb91',
 '1555a01f-0aa7-4429-8a7f-6abf8cfd35ea',
 '36d1f10f-c9b1-44c2-b9b1-bf0f3e1be83d',
 'ef792c0b-25d8-4a84-ac53-8cb9d83a964d',
 'f204aae0-eb6d-444d-aa49-1ecf95048090',
 '223c8a36-ae4f-482e-bfc0-6c3108b2607b',
 '6beb50b4-756d-4a5e-b0bb-a68a74fc9edb',
 '37c5fccb-adf8-4131-946d-2dbb81509b3a',
 '45a7f538-cb12-4175-80ea-35c3097e3372',
 'ec8bdcdd-2193-41a4-a1b8-4a5029b9bda0',
 'b58c32c2-d1c8-44bd-b922-ca7512b7ae1c',
 '55412a9e-480a-4e3a-aebf-401b851c8c0e',
 'ca450234-722e-4d93-803c-6995f23019dd',
 'd0627efc-0967-4c5b-b273-cc9d64ecc2a4',
 'bb554801-bf9f-48b3-90d7-d49d80485bec',
 'a9f980f2-6f84-

In [172]:
print(vector_store.get(where={"personal_id": 1}).get('ids'))

['1ef3a6c8-e4c6-42b8-896a-07ce7cbb32f2', 'b8eba65b-4b7f-46bc-95c6-3b3d28c9dedd']


In [158]:
query = "wer ist scrum master?"
query_embedding = embedding_model.embed_query(query)
results = vector_store.similarity_search(query, k=3)
print(results)

[Document(metadata={'name': 'Martin Wagner'}, page_content='Verantwortlich für die Leitung der agilen Entwicklung und Teamkoordination. Ansprechpartner bei Problemen im agilen Entwicklungsprozess.'), Document(metadata={'name': 'Johannes Schmidt'}, page_content='Führt agile Methoden ein und koordiniert Teams. Ansprechpartner bei Problemen im agilen Entwicklungsprozess.'), Document(metadata={'name': 'Julia Meier'}, page_content='Unterstützt Teams bei der Einführung agiler Methoden. Ansprechpartner bei Coaching und Skalierungsproblemen.')]


In [133]:
# guck in df nach den Mitarbeitern, fasse zusammen
#employees = [df.loc[df["Beschreibung der Position und Zuständigkeiten bei Problemen"] == document.page_content]['Name'].to_string(index=False) for document in results] \
 #           + [df.loc[df["Betreute Programme"] == document.page_content]['Name'].to_string(index=False) for document in results]
#employee_names = employees["Name"]
#print(employees)

['Martin Wagner', 'Johannes Schmidt', 'Julia Neumann\n  Julia Meier', 'Series([], )', 'Series([], )', 'Series([], )']


In [33]:
vector_store.delete_collection()

In [ ]:
def main():
    embedding_model = setup_embedding_model(api_key)
    vector_store = setup_vector_store(api_key)

    query = "wer ist scrum master?"
    results = vector_store.similarity_search(query, k=3)

In [3]:
embedding_model = setup_embedding_model(api_key)
vector_store = setup_vector_store(embedding_model)
summary_chain = get_summary_chain(llm=OpenAI())
output_prompt = get_output_prompt()

/Users/annka/Documents/KI_Hackathon_2024/AIAICaptainHackathonBremen/chatbot/chatbot.py:71: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summary_chain = LLMChain(llm=llm, prompt=summary_prompt)


In [4]:
job = "Scrum Master"
name = "Peter Hase"
job_description = "Er ist Scrum Master"
query = "Ich suche jemand, der Scrum kann"

summary = get_summary(summary_chain, query, name, job, job_description)
output = get_output(output_prompt, name, job, summary)

In [8]:
documents = vector_store.similarity_search(query, k=3)
[document.metadata.get("personal_id") for document in documents]

[61, 3, 2]

In [9]:
print(output)


    Du kannst dich an einen der folgenden Mitarbeiter wenden: 

    Peter Hase, Scrum Master

Peter Hase ist als Scrum Master spezialisiert auf die Anwendung und Umsetzung von Scrum und kann daher bei Fragen und Anliegen zu diesem Thema kompetente Unterstützung bieten.
    


In [5]:
# Definiere den Prompt
prompt_template = PromptTemplate(
    input_variables=["description"],
    #template="Dein Ansprechpartner ist {name}. Position: {position}. Stellenbeschreibung: {description}"
    template="Stellenbeschreibung: {description}"
    #template="Wobei brauchst du Hilfe?"
)

# Erstelle die LLMChain mit dem Prompt
chain = LLMChain(llm=OpenAI(), prompt=prompt_template)

def chatbot():
    print("Wobei brauchst du Hilfe?")
    
    # Hier kannst du die Logik hinzufügen, um die passenden Daten zu bestimmen
    # Beispielwerte
    #description = get_job_description_for_query(user_input, vector_store)

    isRunning = True
    while isRunning:
        user_input = input("> ")
        if user_input == "exit":
            isRunning = False
        else:
            answer = get_job_description_for_query(user_input, vector_store)
            print(answer, '\n')

    # Generiere die Antwort
    #response = chain.invoke(input=description)
    
    #print(response)

In [38]:
chatbot()

Wobei brauchst du Hilfe?
[Document(metadata={'name': 'André Schmidt', 'personal_id': 34}, page_content='VMware, Hyper-V'), Document(metadata={'name': 'Andree Weber', 'personal_id': 39}, page_content='SCCM, Ivanti'), Document(metadata={'name': 'André Schmidt', 'personal_id': 34}, page_content='Verwalten von Servern und Bereitstellung von VMs. Ansprechpartner bei Server- und VM-Problemen.')] 
[Document(metadata={'name': 'Clara Braun', 'personal_id': 57}, page_content=''), Document(metadata={'name': 'Nadine Weber', 'personal_id': 68}, page_content=''), Document(metadata={'name': 'Sarah Wagner', 'personal_id': 102}, page_content='')] 
[Document(metadata={'name': 'Sarah Fischer', 'personal_id': 113}, page_content='SAP'), Document(metadata={'name': 'Vanessa Lang', 'personal_id': 19}, page_content='SAP, Oracle'), Document(metadata={'name': 'Peter Hoffmann', 'personal_id': 6}, page_content='SAP, Procurement Software')] 


In [20]:
from langchain.chains.retrieval_qa.base import RetrievalQA


def retriever_qa_creation():
    llm=OpenAI()
    embedding_model = setup_embedding_model(api_key)
    vector_store = setup_vector_store(embedding_model)
    qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever())
    return qa

rag_chain = retriever_qa_creation()

In [22]:
rag_chain.invoke("VM")

{'query': 'VM',
 'result': '\n\nVM is a virtual machine, a software emulation of a computer system that allows multiple operating systems to run on the same physical hardware.'}